In [1]:
import re
import nltk
import string
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
from nltk.corpus import stopwords
from nltk.stem import SnowballStemmer, WordNetLemmatizer
from sklearn.feature_extraction.text import TfidfVectorizer

from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split

from tensorflow.keras.utils import to_categorical
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences

from tensorflow.keras.optimizers import Adam
from tensorflow.keras.models import Sequential
from tensorflow.keras.callbacks import EarlyStopping
from tensorflow.keras.layers import Dense, LSTM, Embedding, Bidirectional

In [ ]:
import pandas as pd
import numpy as np
train_data = pd.read_csv('/content/drive/MyDrive/ML-Assignment-2/train.csv')

In [ ]:
nltk.download("stopwords")
stop_words = set(stopwords.words("english"))
lemmatizer= WordNetLemmatizer()

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


In [ ]:
# Modelling
from sklearn.model_selection import train_test_split,KFold, GridSearchCV
from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score,confusion_matrix, classification_report
from sklearn.pipeline import Pipeline
from sklearn.metrics import f1_score
from sklearn.svm import SVC

In [ ]:
df_train=train_data
df_train.shape

(630000, 11)

In [ ]:
df_val = df_train.iloc[100001:120000,:]
df_test = df_train.iloc[600001:,:]
df_train = df_train.iloc[:600000,:]

In [ ]:
df_test = df_test[['review_text','rating']]
df_train = df_train[['review_text','rating']]
df_val = df_val[['review_text','rating']]

In [ ]:
#Preprocess text
X_train = df_train['review_text'].values
y_train = df_train['rating'].values

X_test = df_test['review_text'].values
y_test = df_test['rating'].values

In [ ]:
!pip install imblearn

In [ ]:
from imblearn.over_sampling import SMOTE
smote = SMOTE()

In [ ]:
df_train.groupby('rating').describe()


review_text                                         
             count  unique                         top freq
rating                                                     
0            20647   20085  Read through netgalley.com   28
1            19116   19055             Review to come.    7
2            48352   48178             Review to come.   21
3           125891  125139              Review to come   60
4           209227  207500             Review to come.  135
5           176767  175247             Review to come.   94

In [ ]:
import re
import string
from nltk.stem.porter import PorterStemmer
def rem_html(text):
    pattern = re.compile('<.*?>')
    return pattern.sub(r'',text)
def rem_punc(text):
    exclude = string.punctuation
    return text.translate(str.maketrans('','',exclude))
def rem_stopword(text):
    new_text=[]
    for word in text.split():
        if word in stopwords.words('english'):
            new_text.append('')
        else:
            new_text.append(word)
    x = new_text[:]
    new_text.clear()
    return " ".join(x)
def stem_words(text):
    ps = PorterStemmer()
    return " ".join([ps.stem(word) for word in text.split()])

In [ ]:
import re
import string
import nltk
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from nltk.stem import PorterStemmer, WordNetLemmatizer

# Download NLTK resources
nltk.download('punkt')
nltk.download('stopwords')
nltk.download('wordnet')

def preprocess_text(text):
    # Convert to lowercase
    text = text.lower()

    # Remove numbers, punctuation, and special characters
    text = re.sub(r'\d+', '', text)
    text = text.translate(str.maketrans('', '', string.punctuation))

    # Tokenization (split the text into words)
    words = word_tokenize(text)

    # Remove stop words
    stop_words = set(stopwords.words('english'))
    words = [word for word in words if word not in stop_words]

    # Stemming or Lemmatization
    # Choose either stemming or lemmatization based on your preference
    # Stemming (using Porter Stemmer)
    # stemmer = PorterStemmer()
    # words = [stemmer.stem(word) for word in words]

    # Lemmatization (using WordNet Lemmatizer)
    lemmatizer = WordNetLemmatizer()
    words = [lemmatizer.lemmatize(word) for word in words]

    # Join the words back into a single string
    processed_text = ' '.join(words)

    return processed_text

# Example usage:
df_train['review_text']=df_train['review_text'].apply(preprocess_text)


In [ ]:
df_train['review_text']=df_train['review_text'].apply(preprocess_text)
df_test['review_text']=df_test['review_text'].apply(preprocess_text)

In [ ]:
df_train.to_csv("train.csv")

In [2]:
data = pd.read_csv('/content/drive/MyDrive/ML-Assignment-2/pre_data.csv')

In [ ]:
data = data[['review_text','rating']]
data = data.iloc[300000:600000,:]
data.shape

(300000, 2)

In [ ]:
from sklearn.model_selection import train_test_split
X_train,X_test,y_train,y_test = train_test_split(data['review_text'].values.astype('U'),data['rating'].values,test_size=0.3)

In [ ]:
test = pd.read_csv('/content/drive/MyDrive/ML-Assignment-2/Pre_test.csv')

In [ ]:
x_test = test['review_text'].values.astype('U')

In [ ]:
def train_model(model, data, targets):
    """
    Train a model on the given data and targets.

    Parameters:
    model (sklearn model): The model to be trained.
    data (list of str): The input data.
    targets (list of str): The targets.

    Returns:
    Pipeline: The trained model as a Pipeline object.
    """
    # Create a Pipeline object with a TfidfVectorizer and the given model
    text_clf = Pipeline([('vect',TfidfVectorizer()),
                         ('clf', model)])
    # Fit the model on the data and targets
    text_clf.fit(data, targets)
    return text_clf

In [ ]:
def get_F1(trained_model,X,y):
    """
    Get the F1 score for the given model on the given data and targets.

    Parameters:
    trained_model (sklearn model): The trained model.
    X (list of str): The input data.
    y (list of str): The targets.

    Returns:
    array: The F1 score for each class.
    """
    # Make predictions on the input data using the trained model
    predicted=trained_model.predict(X)
    # Calculate the F1 score for the predictions
    f1=f1_score(y,predicted, average=None)
    # Return the F1 score
    return f1

In [ ]:
#Train the model with the training data
log_reg = train_model(LogisticRegression(solver='liblinear',random_state = 0), X_train, y_train)

In [ ]:
#test the model with the test data
y_pred=log_reg.predict(X_test)

#calculate the accuracy
log_reg_accuracy = accuracy_score(y_test, y_pred)
print('Accuracy: ', log_reg_accuracy,'\n')

#calculate the F1 score
f1_Score = get_F1(log_reg,X_test,y_test)
pd.DataFrame(f1_Score, index=data.rating.unique(), columns=['F1 score'])

Accuracy:  0.5074333333333333 



,F1 score
4,0.272583
5,0.218830
2,0.257075
3,0.399894
1,0.535098
0,0.616231


In [ ]:
y_pred = log_reg.predict(x_test)

ValueError: ignored

In [ ]:
##Classification Report
print(classification_report(y_test, y_pred))

              precision    recall  f1-score   support

           0       0.64      0.17      0.27       992
           1       0.54      0.14      0.22       940
           2       0.44      0.18      0.26      2400
           3       0.44      0.37      0.40      6129
           4       0.48      0.61      0.54     10640
           5       0.59      0.65      0.62      8899

    accuracy                           0.51     30000
   macro avg       0.52      0.35      0.38     30000
weighted avg       0.51      0.51      0.49     30000



In [ ]:
#Train the model with the training data
DT = train_model(DecisionTreeClassifier(random_state = 0), X_train, y_train)

#test the model with the test data
y_pred=DT.predict(X_test)

#calculate the accuracy
DT_accuracy = accuracy_score(y_test, y_pred)
print('Accuracy: ', DT_accuracy,'\n')

#calculate the F1 score
f1_Score = get_F1(DT,X_test,y_test)
print(f1_Score)

Accuracy:  0.36441214707156905 

[0.21586616 0.108299   0.15203086 0.30648577 0.40115953 0.45374302]


In [ ]:
#Train the model with the training data
SVM = train_model(SVC(random_state = 0,kernel='rbf',class_weight='balanced'), X_train, y_train)

#test the model with the test data
y_pred=SVM.predict(X_test)

#calculate the accuracy
SVM_accuracy = accuracy_score(y_test, y_pred)
print('Accuracy: ', SVM_accuracy,'\n')

#calculate the F1 score
f1_Score = get_F1(SVM,X_test,y_test)
pd.DataFrame(f1_Score, index=data.rating.unique(), columns=['F1 score'])

In [ ]:
X  = train_data.iloc[:1000,:]

In [ ]:
X = X['review_text'].values

In [ ]:
y = train_data.iloc[:1000,:]

In [ ]:
y = y['rating'].values

In [ ]:
from sklearn.feature_extraction.text import TfidfVectorizer
# Create the TF-IDF vectorizer
tfidf_vectorizer = TfidfVectorizer()

# Fit and transform the corpus to TF-IDF features
tfidf_matrix = tfidf_vectorizer.fit_transform(X)

<1x12705 sparse matrix of type '<class 'numpy.float64'>'
	with 74 stored elements in Compressed Sparse Row format>

In [ ]:
#Train the model with the training data
RF = train_model(RandomForestClassifier(random_state = 0), X_train, y_train)

#test the model with the test data
y_pred=RF.predict(X_test)

#calculate the accuracy
RF_accuracy = accuracy_score(y_test, y_pred)
print('Accuracy: ', RF_accuracy,'\n')

#calculate the F1 score
f1_Score = get_F1(RF, X_test, y_test)
pd.DataFrame(f1_Score, index=df_train.Emotion.unique(), columns=['F1 score'])

Accuracy:  0.4375 



NameError: ignored